In [18]:
import os
from glob import glob
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import minkowski,chebyshev
plt.style.use('ggplot')

In [19]:
def extract_distance_features(df):
    """Returns a DataFrame with columns as the Distances and some geometric measures between two or more Joints."""

    SpineBase = df.iloc[:,0:3].values       #0
    SpineMid = df.iloc[:,3:6].values        #1
    Neck = df.iloc[:,6:9].values            #2
    Head = df.iloc[:,9:12].values           #3
    ShoulderLeft = df.iloc[:,12:15].values  #4
    ElbowLeft = df.iloc[:,15:18].values     #5
    WristLeft = df.iloc[:,18:21].values     #6
    HandLeft = df.iloc[:,21:24].values      #7
    ShoulderRight = df.iloc[:,24:27].values #8
    ElbowRight = df.iloc[:,27:30].values    #9
    WristRight = df.iloc[:,30:33].values    #10
    HandRight = df.iloc[:,33:36].values     #11
    HipLeft = df.iloc[:,36:39].values       #12
    KneeLeft = df.iloc[:,39:42].values      #13
    AnkleLeft = df.iloc[:,42:45].values     #14
    FootLeft = df.iloc[:,45:48].values      #15
    HipRight = df.iloc[:,48:51].values      #16
    KneeRight = df.iloc[:,51:54].values     #17
    AnkleRight = df.iloc[:,54:57].values    #18
    FootRight = df.iloc[:,57:60].values     #19
    SpineShoulder = df.iloc[:,60:63].values #20
    HandTipLeft = df.iloc[:,63:66].values   #21
    ThumbLeft = df.iloc[:,66:69].values     #22
    HandTipRight = df.iloc[:,69:72].values  #23
    ThumbRight = df.iloc[:,72:75].values    #24
    
    points = [SpineBase,SpineMid,Neck,Head,ShoulderLeft,ElbowLeft,WristLeft,HandLeft,ShoulderRight,ElbowRight,WristRight,HandRight,HipLeft,KneeLeft,AnkleLeft,FootLeft,HipRight,KneeRight,AnkleRight,FootRight,SpineShoulder,HandTipLeft,ThumbLeft,HandTipRight,ThumbRight]

    def calculate_distance(v1,v2):
        dist = []
        for i in range(df.shape[0]):
            dist.append(np.sqrt(np.sum(np.square(v1[i] - v2[i])))) ## Euclidean Distance
            # dist.append(np.sum(abs(v1[i]-v2[i]))) ## Manhattan Distance
            # dist.append(minkowski(v1[i],v2[i],p=3)) ## Minkowski Distance
            # dist.append(chebyshev(v1[i],v2[i])) ## Chebyshev Distance
        return dist

    def triangle_area(a,b,c):
        v1 = b-a
        v2 = c-a
        area = []
        for i in range(df.shape[0]):
            area.append(1/2 * (np.linalg.norm(np.cross(v1[i],v2[i]))))
        return area

    def calculate_centroid(a,b,c):
        x_centroid = []
        y_centroid = []
        for i in range(df.shape[0]):
            x_centroid.append((a[i][0] + b[i][0] + c[i][0]) / 3)
            y_centroid.append((a[i][1] + b[i][1] + c[i][1]) / 3)
        return x_centroid, y_centroid

    column = {k:0 for k in range(len(points))}
    print(column)
    count=0
    for i in range(len(points)):
        column[i] = calculate_distance(points[i], points[1])


    column[25] = calculate_distance(points[13],points[17])   ## KneeLeft , KneeRight --> Imp
    column[26] = calculate_distance(points[15],points[19])   ## FootLeft, FootRight  --> Imp
    column[27] = calculate_distance(points[14],points[18])   ## AnkleLeft, AnkleRight --> Imp
    column[28] = calculate_distance(points[15],points[21])   ## FootLeft, HandTipLeft --> Imp
    column[29] = calculate_distance(points[19],points[23])   ## FootRight, HandTipRight -->Imp
    column[30] = calculate_distance(points[7],points[11])    ## HandLeft, HandRight
    column[31] = calculate_distance(points[5],points[9])     ## ElbowLeft, ElbowRight
    column[32] = calculate_distance(points[22],points[13])   ## ThumbLeft, KneeLeft --> Imp
    column[33] = calculate_distance(points[24],points[17])   ## ThumbRight, KneeRight --> Imp
    column[34] = calculate_distance(points[7],points[12])    ## HandLeft, HipLeft
    column[35] = calculate_distance(points[11],points[16])   ## HandRight, HipRight
    column[36] = calculate_distance(points[9],points[16])    ## ElbowRight, HipRight
    column[37] = calculate_distance(points[5],points[12])    ## ElbowLeft, HipLeft

    column[38] = calculate_distance(points[3],points[4])     ## Head, ShoulderLeft
    column[39] = calculate_distance(points[3],points[8])     ## Head, ShoulderRight


    column[40] = triangle_area(points[0],points[15],points[19])   #SpineBase, FootLeft, FootRight -->Imp
    column[41] = triangle_area(points[0],points[14],points[18])   #SpineBase, AnkleLeft, AnkleRight
    column[42] = triangle_area(points[0],points[13],points[17])   #SpineBase, KneeLeft, KneeRight
    column[43] = triangle_area(points[20],points[0],points[23])   #SpineShoulder, SpineBase, HandTipRight --> Imp
    column[44] = triangle_area(points[20],points[0],points[21])   #SpineShoulder, SpineBase, HandTipLeft --> Imp
    column[45] = triangle_area(points[15],points[21],points[20])  #FootLeft, HandTipLeft, SpineShoulder --> Imp
    column[46] = triangle_area(points[19],points[23],points[20])  #FootRight, HandTipRight, SpineShoulder --> Imp
    column[47] = triangle_area(points[15],points[12],points[19])  #FootLeft, HipLeft, FootRight
    column[48] = triangle_area(points[19],points[16],points[15])  #FootRight, HipRight, FootLeft
    column[49] = triangle_area(points[3],points[21],points[12])   #Head, HandTipLeft, HipLeft
    column[50] = triangle_area(points[3],points[23],points[16])   #Head, HandTipRight, HipRight
    column[51] = triangle_area(points[3],points[23],points[19])   #Head, HandTipRight, FootRight --> Imp
    column[52] = triangle_area(points[3],points[21],points[15])   #Head, HandTipLeft, FootLeft --> Imp
    column[53] = triangle_area(points[5],points[21],points[12])   #ElbowLeft, HandTipLeft, HipLeft
    column[54] = triangle_area(points[9],points[23],points[16])   #ElbowRight, HandTipRight, HipRight
    column[55] = triangle_area(points[20],points[5],points[1])    #SpineShoulder, ElbowLeft, SpineMid
    column[56] = triangle_area(points[20],points[9],points[1])    #SpineShoulder, ElbowRight, SpineMid
    
    column[57], column[58] = calculate_centroid(points[0],points[15],points[19])
    column[59], column[60] = calculate_centroid(points[0],points[14],points[18])
    column[61], column[62] = calculate_centroid(points[0],points[13],points[17])
    column[63], column[64] = calculate_centroid(points[20],points[0],points[23])
    column[65], column[66] = calculate_centroid(points[20],points[0],points[21])
    column[67], column[68] = calculate_centroid(points[19],points[23],points[20])
    column[69], column[70] = calculate_centroid(points[15],points[21],points[20])
    column[71], column[72] = calculate_centroid(points[19],points[16],points[15])
    column[73], column[74] = calculate_centroid(points[3],points[21],points[12])
    column[75], column[76] = calculate_centroid(points[3],points[23],points[16])
    column[77], column[78] = calculate_centroid(points[3],points[23],points[19])
    column[79], column[80] = calculate_centroid(points[3],points[21],points[15])
    column[81], column[82] = calculate_centroid(points[5],points[21],points[12])
    column[83], column[84] = calculate_centroid(points[9],points[23],points[16])
    column[85], column[86] = calculate_centroid(points[20],points[5],points[1])
    column[87], column[88] = calculate_centroid(points[20],points[9],points[1])
    column[89], column[90] = calculate_centroid(points[15],points[12],points[19])
    
    column.__delitem__(1)

    return pd.DataFrame(column)


In [20]:
df = pd.read_csv(r"D:\Research_Project\My_project_22\input\raw\Both Hands front UP Left Leg Front UP\aatish_both_hand_frontup_left_leg_frontup_trial1_interpolated.csv",header=None)
df = df.iloc[:,1:]

In [21]:
x = extract_distance_features(df)
x

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}


,0,2,3,4,5,6,7,8,9,10,...,81,82,83,84,85,86,87,88,89,90
0,0.324883,0.314224,0.475169,0.257537,0.224666,0.380845,0.480537,0.253687,0.224452,0.390823,...,0.213617,-0.038593,0.540860,-0.051324,0.304090,0.305887,0.446047,0.299417,0.346157,-0.625023
1,0.322438,0.312178,0.470942,0.260437,0.229644,0.384718,0.450586,0.252057,0.222488,0.390271,...,0.215780,-0.039860,0.542580,-0.049387,0.308230,0.300970,0.449470,0.296057,0.350710,-0.623807
2,0.321594,0.311465,0.469381,0.261410,0.231567,0.386289,0.442863,0.251537,0.221754,0.390644,...,0.217133,-0.040974,0.544257,-0.049518,0.309700,0.299360,0.450647,0.295027,0.352113,-0.623471
3,0.321504,0.311379,0.469090,0.261484,0.231913,0.386717,0.444212,0.251446,0.221688,0.390763,...,0.217390,-0.041400,0.545180,-0.049867,0.309900,0.299230,0.450833,0.294923,0.352107,-0.623497
4,0.321446,0.311360,0.469092,0.261522,0.232146,0.387123,0.445079,0.251438,0.222014,0.390721,...,0.216603,-0.041421,0.544767,-0.049813,0.309990,0.299090,0.451043,0.294643,0.352097,-0.623521
5,0.321372,0.311293,0.468874,0.261605,0.232289,0.391229,0.452269,0.251313,0.222774,0.393711,...,0.216057,-0.042896,0.544937,-0.051748,0.310097,0.299247,0.451297,0.294457,0.351893,-0.623703
6,0.321137,0.311042,0.468723,0.261924,0.232183,0.402413,0.460417,0.251269,0.222349,0.394016,...,0.215777,-0.043086,0.546767,-0.051293,0.310387,0.299377,0.451470,0.294390,0.351900,-0.623696
7,0.321111,0.311054,0.468740,0.262211,0.230893,0.414605,0.471387,0.251299,0.222548,0.397719,...,0.215297,-0.045022,0.547710,-0.052670,0.310843,0.300107,0.451600,0.294383,0.351997,-0.623726
8,0.321003,0.310997,0.468768,0.262697,0.229234,0.426640,0.497247,0.251247,0.227386,0.415595,...,0.213537,-0.047941,0.551587,-0.055973,0.312220,0.302663,0.453307,0.294450,0.352190,-0.623674
9,0.320646,0.310729,0.468664,0.262883,0.231696,0.432914,0.511073,0.251095,0.233253,0.426789,...,0.213343,-0.042298,0.558297,-0.046964,0.314120,0.305107,0.455597,0.298010,0.352567,-0.623908


In [22]:
def extract_angle_features(df):
    """Returns a DataFrame with columns as the angles extracted from 3 joints."""
    SpineBase = df.iloc[:,0:3].values       #0
    SpineMid = df.iloc[:,3:6].values        #1
    Neck = df.iloc[:,6:9].values            #2
    Head = df.iloc[:,9:12].values           #3
    ShoulderLeft = df.iloc[:,12:15].values  #4
    ElbowLeft = df.iloc[:,15:18].values     #5
    WristLeft = df.iloc[:,18:21].values     #6
    HandLeft = df.iloc[:,21:24].values      #7
    ShoulderRight = df.iloc[:,24:27].values #8
    ElbowRight = df.iloc[:,27:30].values    #9
    WristRight = df.iloc[:,30:33].values    #10
    HandRight = df.iloc[:,33:36].values     #11
    HipLeft = df.iloc[:,36:39].values       #12
    KneeLeft = df.iloc[:,39:42].values      #13
    AnkleLeft = df.iloc[:,42:45].values     #14
    FootLeft = df.iloc[:,45:48].values      #15
    HipRight = df.iloc[:,48:51].values      #16
    KneeRight = df.iloc[:,51:54].values     #17
    AnkleRight = df.iloc[:,54:57].values    #18
    FootRight = df.iloc[:,57:60].values     #19
    SpineShoulder = df.iloc[:,60:63].values #20
    HandTipLeft = df.iloc[:,63:66].values   #21
    ThumbLeft = df.iloc[:,66:69].values     #22
    HandTipRight = df.iloc[:,69:72].values  #23
    ThumbRight = df.iloc[:,72:75].values    #24
    
    def calculate_angle(a1,a2,a3):
        v1 = a1 - a2
        v2 = a3 - a2
        angle = []

        #### Angle Calculation Formula 1
        # for i in range(df.shape[0]):
        #     unit_vector1 = v1[i] / np.linalg.norm(v1[i])
        #     unit_vector2 = v2[i] / np.linalg.norm(v2[i])

        #     dot_product = np.dot(unit_vector1, unit_vector2)

        #     angle.append(np.arccos(dot_product)) #angle in radian


        #### Angle Calculation Formula 2
        for i in range(df.shape[0]):
            res = (np.arctan(np.linalg.norm(np.cross(v1[i],v2[i])) / np.dot(v1[i],v2[i])) / np.pi) * 180
            angle.append(res)

        return angle

    angle_1 = calculate_angle(ShoulderLeft,ElbowLeft,WristLeft)
    angle_2 = calculate_angle(ShoulderRight,ElbowRight,WristRight)
    angle_3 = calculate_angle(Head,ShoulderLeft,HandLeft)
    angle_4 = calculate_angle(Head,ShoulderRight,HandRight)
    angle_5 = calculate_angle(SpineShoulder,ElbowLeft,ThumbLeft)
    angle_6 = calculate_angle(SpineShoulder,ElbowRight,ThumbRight)
    angle_7 = calculate_angle(SpineBase,KneeLeft,FootLeft)
    angle_8 = calculate_angle(SpineBase,KneeRight,FootRight)
    angle_9 = calculate_angle(SpineMid,HipLeft,AnkleLeft)
    angle_10 = calculate_angle(SpineMid,HipRight,AnkleRight)
    angle_11 = calculate_angle(ElbowLeft,ShoulderLeft,SpineShoulder)
    angle_12 = calculate_angle(ElbowRight,ShoulderRight,SpineShoulder)
    angle_13 = calculate_angle(ElbowRight,ShoulderRight,SpineMid)
    angle_14 = calculate_angle(ElbowLeft,ShoulderLeft,SpineMid)
    angle_15 = calculate_angle(Head,ShoulderLeft,ElbowLeft)
    angle_16 = calculate_angle(Head,ShoulderRight,ElbowRight)
    angle_17 = calculate_angle(ShoulderLeft,HipLeft,KneeLeft)
    angle_18 = calculate_angle(ShoulderRight,HipRight,KneeRight)
    angle_19 = calculate_angle(SpineBase,HipRight,KneeRight)
    angle_20 = calculate_angle(SpineBase,HipLeft,KneeLeft)
    angle_21 = calculate_angle(Neck,ShoulderLeft,ElbowLeft)
    angle_22 = calculate_angle(Neck,ShoulderRight,ElbowRight)
    angle_23 = calculate_angle(KneeLeft,AnkleLeft,FootLeft)
    angle_24 = calculate_angle(KneeRight,AnkleRight,FootRight)
    
    
    angle_df = pd.DataFrame(data=[angle_1,angle_2,angle_3,angle_4,angle_5,angle_6,angle_7,angle_8,angle_9,angle_10,angle_11,angle_12,angle_13,angle_14,angle_15,angle_16,angle_17,angle_18,angle_19,angle_20,angle_21,angle_22,angle_23,angle_24]).T
    
    return angle_df

In [28]:
x = extract_distance_features(df)
x

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}


,0,2,3,4,5,6,7,8,9,10,...,81,82,83,84,85,86,87,88,89,90
0,0.324883,0.314224,0.475169,0.257537,0.224666,0.380845,0.480537,0.253687,0.224452,0.390823,...,0.213617,-0.038593,0.540860,-0.051324,0.304090,0.305887,0.446047,0.299417,0.346157,-0.625023
1,0.322438,0.312178,0.470942,0.260437,0.229644,0.384718,0.450586,0.252057,0.222488,0.390271,...,0.215780,-0.039860,0.542580,-0.049387,0.308230,0.300970,0.449470,0.296057,0.350710,-0.623807
2,0.321594,0.311465,0.469381,0.261410,0.231567,0.386289,0.442863,0.251537,0.221754,0.390644,...,0.217133,-0.040974,0.544257,-0.049518,0.309700,0.299360,0.450647,0.295027,0.352113,-0.623471
3,0.321504,0.311379,0.469090,0.261484,0.231913,0.386717,0.444212,0.251446,0.221688,0.390763,...,0.217390,-0.041400,0.545180,-0.049867,0.309900,0.299230,0.450833,0.294923,0.352107,-0.623497
4,0.321446,0.311360,0.469092,0.261522,0.232146,0.387123,0.445079,0.251438,0.222014,0.390721,...,0.216603,-0.041421,0.544767,-0.049813,0.309990,0.299090,0.451043,0.294643,0.352097,-0.623521
5,0.321372,0.311293,0.468874,0.261605,0.232289,0.391229,0.452269,0.251313,0.222774,0.393711,...,0.216057,-0.042896,0.544937,-0.051748,0.310097,0.299247,0.451297,0.294457,0.351893,-0.623703
6,0.321137,0.311042,0.468723,0.261924,0.232183,0.402413,0.460417,0.251269,0.222349,0.394016,...,0.215777,-0.043086,0.546767,-0.051293,0.310387,0.299377,0.451470,0.294390,0.351900,-0.623696
7,0.321111,0.311054,0.468740,0.262211,0.230893,0.414605,0.471387,0.251299,0.222548,0.397719,...,0.215297,-0.045022,0.547710,-0.052670,0.310843,0.300107,0.451600,0.294383,0.351997,-0.623726
8,0.321003,0.310997,0.468768,0.262697,0.229234,0.426640,0.497247,0.251247,0.227386,0.415595,...,0.213537,-0.047941,0.551587,-0.055973,0.312220,0.302663,0.453307,0.294450,0.352190,-0.623674
9,0.320646,0.310729,0.468664,0.262883,0.231696,0.432914,0.511073,0.251095,0.233253,0.426789,...,0.213343,-0.042298,0.558297,-0.046964,0.314120,0.305107,0.455597,0.298010,0.352567,-0.623908


#### CREATING EXTRACTED FEATURES CSV

In [24]:
INPUT_PATH = r'D:\Research_Project\My_project_22\input\raw'
ANGLES_PATH = r'D:\Research_Project\My_project_22\FEATURES_EXTRACTED\ANGLES'
DISTANCES_PATH = r"D:\Research_Project\My_project_22\FEATURES_EXTRACTED\DISTANCES"

In [25]:
raw_input_files = glob(os.path.join(INPUT_PATH,"*","*_interpolated.csv"))
print(raw_input_files)

['D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands front UP Left Leg Front UP\\aatish_both_hand_frontup_left_leg_frontup_trial1_interpolated.csv', 'D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands front UP Left Leg Front UP\\aatish_both_hand_frontup_left_leg_frontup_trial2_interpolated.csv', 'D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands front UP Left Leg Front UP\\arnab_both_hand_frontup_left_leg_frontup_trial1_interpolated.csv', 'D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands front UP Left Leg Front UP\\arnab_both_hand_frontup_left_leg_frontup_trial2_interpolated.csv', 'D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands front UP Left Leg Front UP\\Harsh_both_hand_frontup_left_leg_frontup_trial1_interpolated.csv', 'D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands front UP Left Leg Front UP\\Harsh_both_hand_frontup_left_leg_frontup_trial2_interpolated.csv', 'D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands 

In [26]:
def save_csv_dir(x,path):
    file_name_splits = os.path.basename(x).split('_')
    trial_string_idx = [i for i, elem in enumerate(file_name_splits) if re.search('trial*', elem)][0]
    gesture = "_".join(file_name_splits[1:trial_string_idx])

    save_dir = os.path.join(path,gesture)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    return save_dir

def extract_participant_pose_name(x):
    x = x.split("\\")[-1]
    return x

In [27]:
for i in range(len(raw_input_files)):
    data =pd.read_csv(raw_input_files[i],header=None)
    data = data.iloc[:,1:]
    folder_name = extract_participant_pose_name(raw_input_files[i])
    # print(folder_name)
    distance_df = extract_distance_features(data)
    angle_df = extract_angle_features(data)
    angles_save_dir = save_csv_dir(raw_input_files[i],ANGLES_PATH)
    distances_save_dir = save_csv_dir(raw_input_files[i],DISTANCES_PATH)
    angles_save_path = os.path.join(angles_save_dir,folder_name)
    distances_save_path = os.path.join(distances_save_dir,folder_name)
    distance_df.to_csv(distances_save_path,index=None,header=None)
    angle_df.to_csv(angles_save_path,index=None,header=None)
        

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}
{0: 

In [24]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Model configuration
additional_metrics = ['accuracy']
batch_size = 128
embedding_output_dims = 15
loss_function = BinaryCrossentropy()
max_sequence_length = 300
num_distinct_words = 5000
number_of_epochs = 5
optimizer = Adam()
validation_split = 0.20
verbosity_mode = 1

# Disable eager execution
tf.compat.v1.disable_eager_execution()

# Load dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_distinct_words)
print(type(x_train))
print(type(y_train))
print(type(x_test))
print(type(y_test))
print(x_train.shape)
print(x_test.shape)
print(x_train.head(10))

# # Pad all sequences
# padded_inputs = pad_sequences(x_train, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>
# padded_inputs_test = pad_sequences(x_test, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>

# # Define the Keras model
# model = Sequential()
# model.add(Embedding(num_distinct_words, embedding_output_dims, input_length=max_sequence_length))
# model.add(LSTM(10))
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(optimizer=optimizer, loss=loss_function, metrics=additional_metrics)

# # Give a summary
# model.summary()

# # Train the model
# history = model.fit(padded_inputs, y_train, batch_size=batch_size, epochs=number_of_epochs, verbose=verbosity_mode, validation_split=validation_split)

# # Test the model after training
# test_results = model.evaluate(padded_inputs_test, y_test, verbose=False)
# print(f'Test results - Loss: {test_results[0]} - Accuracy: {100*test_results[1]}%')

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(25000,)
(25000,)


AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [21]:

import config
raw_angle_files_1 = glob(os.path.join(config.INPUT_DISTANCES, "*", "*.csv"))
file_name_splits = os.path.basename(raw_angle_files_1[0]).split('_')
print(file_name_splits)


ModuleNotFoundError: No module named 'config'